In [169]:
import pandas as pd
import requests
from datetime import datetime
from math import ceil
from urllib.parse import urlencode
import plotly.express as px
import aiohttp
import asyncio
import nest_asyncio

### Vamos analisar o payload da busca "A receber/recebendo proposta"

<img src="assets/img/recebendo-proposta.jpg" width="600" alt="busca-recebendo-proposta"/>
<img src="assets/img/payload-recebendo-proposta.jpg" width="600" alt="payload-recebendo-proposta"/>
<br></br>
<img src="assets/img/preview-recebendo-proposta.jpg" width="600" alt="preview-recebendo-proposta"/>


### Observamos no preview que a API nos fornece em "filters" diversas informações úteis como a quantidade de licitações "recebendo proposta" por ano, esferas, modalidade, municípios, orgão, poderes, tipos, tipo de contrato, estado e os 10 itens constantes na página sendo visualizada, além de o total geral de licitações recebendo propostas.

### Podemos gerar alguns gráficos para visualizarmos melhor algumas informações, tais como:

- Quantidade de licitações recebendo propostas por estado;
- Mapa da quantidade de licitações recebendo propostas por ano;
- Quantidade de licitações recebendo propostas por modalidade;
- Quantidade de licitações recebendo propostas por tipo;


#### Vamos criar as funções necessárias.

In [170]:
def request_editais(editais_params):

    r = requests.get('https://pncp.gov.br/api/search/', params=editais_params)
    return r

#Criando funções assíncronas para realizar o carregamento de múltiplas páginas em paralelo

async def fetch_multiple(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)
    return responses

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.json()

async def main(urls):
    item =[]
    urls = urls
    responses = await fetch_multiple(urls)
    for response in responses:
        item.append(response)
    return item

In [171]:
params = {
    'tipos_documento': 'edital',
    'ordenacao': '-data',
    'pagina': '1',
    'tam_pagina': '10',
    'status': 'recebendo_proposta',
}

r = requests.get('https://pncp.gov.br/api/search/', params=params)
data = r.json()

In [172]:
licit_anos = pd.DataFrame(data['filters']['anos'])
licit_esferas = pd.DataFrame(data['filters']['esferas'])[['nome', 'total']]
licit_modalidades = pd.DataFrame(data['filters']['modalidades'])
licit_municipios = pd.DataFrame(data['filters']['municipios'])[['nome', 'total']]
licit_orgaos = pd.DataFrame(data['filters']['orgaos'])
licit_poderes = pd.DataFrame(data['filters']['poderes'])
licit_tipo = pd.DataFrame(data['filters']['tipos'])
licit_ufs = pd.DataFrame(data['filters']['ufs'])
licit_unidades = pd.DataFrame(data['filters']['unidades'])[['nome', 'total']]

In [173]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Initialize figure with subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=("Licitações recebendo propostas por ano", "Licitações recebendo propostas por estado",
                                                "Licitações recebendo propostas por modalidade", "Licitações recebendo propostas por tipo"))

# Add traces

#Licitações por ano
licit_anos = licit_anos.sort_values(by='ano')
fig.add_trace(go.Bar(x=licit_anos['ano'], y=licit_anos['total']), row=1, col=1),

#Licitações por estado
fig.add_trace(go.Bar(x=licit_ufs['id'], y=licit_ufs['total']), row=1, col=2)

#Licitações por modalidade
fig.add_trace(go.Bar(x=licit_modalidades['nome'], y=licit_modalidades['total']), row=2, col=1)

#Licitações por tipo
fig.add_trace(go.Bar(x=licit_tipo['nome'], y=licit_tipo['total']), row=2, col=2)

# Update title and height
fig.update_layout(showlegend=False, height=700)

fig.show()

#### Vamos criar um mapa do Brasil exibindo a quantidade de licitações por cidade, mas primeiro precisamos adicionar uma coluna com o total de licitações para cada munícipio na tabela "coordenadas"

In [174]:
#Obtendo coordenadas dos municípios
coordenadas = pd.read_csv('lat-long-municipios.csv', sep=',')
coordenadas.head()

,municipio,latitude,longitude
0,Abadia de Goiás,-16.75730,-49.4412
1,Abadia dos Dourados,-18.48310,-47.3916
2,Abadiânia,-16.19700,-48.7057
3,Abaeté,-19.15510,-45.4444
4,Abaetetuba,-1.72183,-48.8788


In [175]:
licit_municipios.rename(columns={'nome':'municipio'}, inplace=True)
licit_municipios_lat_long = licit_municipios.merge(coordenadas, how='inner', on='municipio')
licit_municipios_lat_long

,municipio,total,latitude,longitude
0,Rio de Janeiro,396,-22.9129,-43.2003
1,Brasília,335,-15.7795,-47.9297
2,São Paulo,330,-23.5329,-46.6395
3,Florianópolis,211,-27.5945,-48.5477
4,Curitiba,193,-25.4195,-49.2646
...,...,...,...,...
1090,Tremembé,5,-22.9571,-45.5475
1091,Ampére,5,-25.9168,-53.4686
1092,Bituruna,5,-26.1607,-51.5518
1093,Cafeara,5,-22.7890,-51.7142


In [176]:
fig = px.scatter_mapbox(licit_municipios_lat_long, lat="latitude", lon="longitude", hover_name="municipio", hover_data="total", size="total",
                        color_discrete_sequence=["fuchsia"], zoom=3, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Uma das coisas mais importantes para quem pesquisa no portal é conseguir ver quais items estão sendo licitados por edital sem ter que abrir cada um.
#### Por exemplo, seria interessante termos uma tabela indicando diretamente quais itens e os valores que estão sendo pedidos por edital.

In [177]:
params_editais = {
        'tipos_documento': 'edital',
        'ordenacao': '-data',
        'pagina': '1',
        'tam_pagina': '10',
        'status': 'recebendo_proposta',
        'municipios': 4359
    }

r = request_editais(params_editais)

total_items = r.json()['total']
total_paginas = ceil(total_items/int(params_editais['tam_pagina']))
total_paginas

3

In [178]:
editais = []
items = []
df_editais = pd.DataFrame()
df_items = pd.DataFrame()

params_items = {
    'páginas': 1,
    'tamanhoPagina': 10,
    'municipios': 4359
}

for pagina in range(1,total_paginas + 1):
    
    params_editais['pagina'] = pagina
    registros_editais = pd.json_normalize(request_editais(params_editais).json()['items'])[['municipio_nome','esfera_nome', 'uf', 'orgao_nome', 'orgao_cnpj' ,'description', 'modalidade_licitacao_nome',
                                              'data_inicio_vigencia', 'data_fim_vigencia', 'ano', 'numero_sequencial']]
    editais.append(registros_editais)
    for edital in range(len(registros_editais)):
        orgao_cnpj = registros_editais['orgao_cnpj'][edital] #Colocar o numerp da linha do df_editais
        ano_url = registros_editais['ano'][edital] # conferir para ver se realmente o ano que consta na url
        numero_sequencial = registros_editais['numero_sequencial'][edital]

        items_qtd = requests.get('https://pncp.gov.br/api/pncp/v1/orgaos/' + orgao_cnpj + '/compras/' + ano_url + '/' + numero_sequencial +'/itens/quantidade').json()
        total_paginas_items = ceil(items_qtd / params_items['tamanhoPagina'])
    
        for pagina_item in range(1,total_paginas_items + 1):

            params_items['páginas'] = pagina_item

            r_items = requests.get('https://pncp.gov.br/api/pncp/v1/orgaos/' + orgao_cnpj +'/compras/' + ano_url + '/' +  numero_sequencial + '/itens?', params=params_items)

            registro_items = pd.json_normalize(r_items.json()) \
                [['descricao', 'materialOuServicoNome', 'quantidade', 'unidadeMedida', 'valorUnitarioEstimado', 'valorTotal', 'orcamentoSigiloso', 'tipoBeneficioNome']]
            registro_items['Numero_Sequencial'] = numero_sequencial
            registro_items['Orgao_CNPJ'] = orgao_cnpj
            registro_items['Ano'] = ano_url

            items.append(registro_items)
    
df_items = pd.concat(items, ignore_index=True)
df_items.rename(columns = {'descricao':'Descricao_Item','materialOuServicoNome':'Tipo','quantidade':'Qtd','unidadeMedida':'Un. Medida',
                            'valorUnitarioEstimado':'Valor_Unit.', 'valorTotal':'Valor_Total', 'orcamentoSigiloso':'Sigiloso',
                            'tipoBeneficioNome':'Tipo_Beneficio'}, inplace = True)


df_editais = pd.concat(editais, ignore_index=True)
df_editais.rename(columns = {'numero_sequencial':'Numero_Sequencial', 'municipio_nome':'Municipio', 'esfera_nome':'Esfera', 'orgao_nome':'Orgao', 'orgao_cnpj':'Orgao_CNPJ', 'description':'Descricao',
                              'modalidade_licitacao_nome':'Modalidade', 'data_inicio_vigencia':'Data_Inicio',
                                'data_fim_vigencia':'Data_Fim', 'ano':'Ano', 'uf': 'UF'}, inplace = True)


df_compilado = df_items.merge(df_editais, left_on=['Ano', 'Numero_Sequencial', 'Orgao_CNPJ'], right_on=['Ano', 'Numero_Sequencial', 'Orgao_CNPJ'])
df_compilado.set_index(['Ano', 'Numero_Sequencial', 'Orgao_CNPJ'])

Descricao_Item  \
Ano  Numero_Sequencial Orgao_CNPJ                                                          
2024 27                83102343000194  RAÇÃO CANINA SUPER PREMIUM PARA CÃES IDOSOS, D...   
                       83102343000194  ALIMENTO COADJUVANTE ÚMIDO PARA CÃES ADULTOS. ...   
                       83102343000194  SNACKE CROCANTE (PETISCO) PARA CÃES, PORTE MÉD...   
                       83102343000194  COLEIRA,  CINZENTA, INODORA ANTI-PULGAS.  SUBS...   
                       83102343000194  SHAMPOO ANTIPULGAS-PERMETRINA 1%, INDICADO PAR...   
...                                                                                  ...   
2023 13                11188015000141  GRUPO 02 PROCEDIMENTOS COM FINALIDADES DIAGNÓS...   
     3                 04894677000171                              AVALIACAO DE PROJETOS   
     7                 11188015000141  GRUPO 07 ÓRTESES, PRÓTESES E MATERIAIS ESPECIA...   
     1                 04894677000171  PRESTAÇÃO DE SERVIÇOS DE PROFISSIONAIS DE ARTE...   
     3                 11188015000141  SERVIÇOS DE SAÚDE NA ESPECIALIDADE DE OFTALMOL...   

                                           Tipo    Qtd      Un. Medida  \
Ano  Numero_Sequencial Orgao_CNPJ                                        
2024 27                83102343000194  Material  300.0           Quilo   
                       83102343000194  Material   60.0         Unidade   
                       83102343000194  Material   12.0         Unidade   
                       83102343000194  Material    6.0         Unidade   
                       83102343000194  Material    6.0         Unidade   
...                                         ...    ...             ...   
2023 13                11188015000141  Material    1.0    UNIDADE (UN)   
     3                 04894677000171  Material    1.0    UNIDADE (UN)   
     7                 11188015000141  Material   12.0    UNIDADE (UN)   
     1                 04894677000171  Material    1.0   SERV. (SERV.)   
     3                 11188015000141   Serviço    1.0  UNIDADE (UND.)   

                                       Valor_Unit.  Valor_Total  Sigiloso  \
Ano  Numero_Sequencial Orgao_CNPJ                                           
2024 27                83102343000194        26.49      7947.00     False   
                       83102343000194         2.99       179.40     False   
                       83102343000194        27.92       335.04     False   
                       83102343000194       229.83      1378.98     False   
                       83102343000194        31.93       191.58     False   
...                                            ...          ...       ...   
2023 13                11188015000141   2980800.00   2980800.00     False   
     3                 04894677000171     30000.00     30000.00     False   
     7                 11188015000141     14250.00    171000.00     False   
     1                 04894677000171     60000.00     60000.00     False   
     3                 11188015000141   1200000.00   1200000.00     False   

                                      Tipo_Beneficio Municipio     Esfera  UF  \
Ano  Numero_Sequencial Orgao_CNPJ                                               
2024 27                83102343000194  Sem benefício   Brusque  Municipal  SC   
                       83102343000194  Sem benefício   Brusque  Municipal  SC   
                       83102343000194  Sem benefício   Brusque  Municipal  SC   
                       83102343000194  Sem benefício   Brusque  Municipal  SC   
                       83102343000194  Sem benefício   Brusque  Municipal  SC   
...                                              ...       ...        ...  ..   
2023 13                11188015000141  Não se aplica   Brusque  Municipal  SC   
     3                 04894677000171  Não se aplica   Brusque  Municipal  SC   
     7                 11188015000141  Não se aplica   Brusque  Municipal  SC   
     1               

#### No entanto essa operação exige o request de diversas páginas ao servidor, o que leva um tempo considerável se não for realizado em paralelo, como pode ser observado acima.

### Testando request paralelizado

#### TODO: criar listas com os links dos editais e outra lista com os links de cada página de cada edital;

In [ ]:
nest_asyncio.apply()
loop = asyncio.get_event_loop()

links_editais = []
lista_editais = []
df_editais = pd.DataFrame()

params_editais = {
        'q':'',
        'tipos_documento': 'edital',
        'ordenacao': '-data',
        'tam_pagina': '10',
        'status': 'recebendo_proposta',
        'municipios': 4006,
        'pagina': '1',
    }

r = request_editais(params_editais)

total_items = r.json()['total']
total_paginas = ceil(total_items/int(params_editais['tam_pagina']))
total_paginas

url_base = 'https://pncp.gov.br/api/search/?'

for pagina in range(1,total_paginas + 1):
    
    params_editais['pagina'] = pagina
    links_editais.append(url_base +  urlencode(params_editais))
    
editais = loop.run_until_complete(main(links_editais))

# for i in range(0,len(editais)): #limitado pelo numero de páginas suportados no site de 999
#   lista_editais.append(editais[i]['items'])

# lista_df_editais = [pd.json_normalize(lista_editais[i]) for i in range(len(lista_editais))]
# df_editais = pd.concat(lista_df_editais, ignore_index=True)
# df_editais

In [ ]:
link_editais =[]
partes_do_link = editais[0]['items'][0]['item_url'].split('/')
link_editais = 'https://pncp.gov.br/api/pncp/v1/orgaos/' +  partes_do_link[2] + '/compras/' + partes_do_link[3] + '/' + partes_do_link[4]
link_editais

In [ ]:
links_total_items_edital = ['https://pncp.gov.br/api/pncp/v1/orgaos/'+ url.split('/')[2] + '/compras/' + url.split('/')[3] + '/' + url.split('/')[4] + '/itens/quantidade' for url in df_editais['item_url']]

In [ ]:
links_total_items_edital

In [ ]:
nest_asyncio.apply()
loop = asyncio.get_event_loop()

total_items_edital = loop.run_until_complete(main(links_total_items_edital))
total_items_edital

In [ ]:
params_items = {
    'páginas': '1',
    'tamanhoPagina': '10'
}

link_itens_edital = []
paginas_items_edital = [ceil(int(x) / int(params_items['tamanhoPagina'])) for x in total_items_edital]
for item in range(len(links_total_items_edital)):
  if paginas_items_edital[item] > 1:
    link = [links_total_items_edital[item-1][:-11] + '?pagina=' + str(i) + '&tamanhoPagina=' + params_items['tamanhoPagina'] for i in range(1,paginas_items_edital[item] +1)]
    link_itens_edital.extend(link)
  else: link_itens_edital.append(links_total_items_edital[item-1][:-11] + '?pagina=' + str(1) + '&tamanhoPagina=' + params_items['tamanhoPagina'])

In [ ]:
link_itens_edital

In [ ]:
total_items_todos_edital = loop.run_until_complete(main(link_itens_edital))
total_items_todos_edital

In [ ]:
df=pd.DataFrame()
for i in range(len(total_items_todos_edital)):
  
  df = pd.concat([df, pd.json_normalize(total_items_todos_edital[i])], ignore_index=True)
df.unstack()
df

In [ ]:
params_editais = {
        'tipos_documento': 'edital',
        'ordenacao': '-data',
        'pagina': '1',
        'tam_pagina': '10',
        'status': 'recebendo_proposta',
        'municipios': None
    }

filters = request_editais(params_editais).json()['filters']
columns = pd.json_normalize(request_editais(params_editais).json()['filters']).columns
df_filters = []


for column in columns:
  df_filters.append(pd.json_normalize(filters[column]))

df_filters[5]